In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt



In [2]:
locA_solar_sensor = []
locB_solar_sensor = []

for i in range(1,45):
    try:
        t = pd.read_csv(f"../Datasets/Anti_Anomaly_data/Anti_Anomaly_A{i}_df.csv")
        t['DATE_TIME'] = pd.to_datetime(t['DATE_TIME'])
        locA_solar_sensor.append(t)
    except Exception as e:
        locA_solar_sensor.append([])
    try:
        t = pd.read_csv(f"../Datasets/Anti_Anomaly_data/Anti_Anomaly_B{i}_df.csv")
        t['DATE_TIME'] = pd.to_datetime(t['DATE_TIME'])
        locB_solar_sensor.append(t)
    except Exception as e:
        locB_solar_sensor.append([])


In [3]:
final_locA = []
final_locB = []

def preprocess_data(locA_solar_sensor,locB_solar_sensor):
    locA_models = []
    locB_models = []

    for i in range(len(locA_solar_sensor)):
        if len(locA_solar_sensor[i]) > 0:

            current_sensor = locA_solar_sensor[i].copy()
            # Add season and time period columns
            current_sensor.loc[:, 'SEASON'] = current_sensor['DATE_TIME'].dt.month.map({
                12: 1, 1: 1, 2: 1,
                3: 2, 4: 2, 5: 2,
                6: 3, 7: 3, 8: 3,
                9: 4, 10: 4, 11: 4})

            current_sensor.loc[:, 'TIME_PERIOD'] = pd.cut(current_sensor['DATE_TIME'].dt.hour,
                                                    bins=[0, 6, 12, 15, 20, 24],
                                                    labels=[1, 2, 3, 4, 5],
                                                    include_lowest=True)

            # Create dummy variables
            season_encoded = pd.get_dummies(current_sensor['SEASON'].astype(str), prefix='Season')
            time_period_encoded = pd.get_dummies(current_sensor['TIME_PERIOD'].astype(str), prefix='TimePeriod')
            current_sensor = pd.concat([current_sensor, season_encoded, time_period_encoded], axis=1)
            current_sensor = current_sensor.drop(['SEASON', 'TIME_PERIOD'], axis=1)

            # Prepare features and target
            X = current_sensor[['Conversion_Efficiency','Normalized_Power','Energy_Per_Irradiation','AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION'] +
                        list(season_encoded.columns) + list(time_period_encoded.columns)]
            y = current_sensor['AC_POWER']

            final_locA.append([X, y])
        else:
            final_locA.append([[],[]])

    print("*************************************","Location A: ",len(final_locA))
    # Process Location B sensors
    for i in range(len(locB_solar_sensor)):
        if len(locB_solar_sensor[i])>0:

            current_sensor = locB_solar_sensor[i].copy()
            # Add season and time period columns
            current_sensor.loc[:,'SEASON'] = current_sensor['DATE_TIME'].dt.month.map({12:1, 1:1, 2:1,
                                                            3:2, 4:2, 5:2,
                                                            6:3, 7:3, 8:3, 
                                                            9:4, 10:4, 11:4})

            current_sensor.loc[:,'TIME_PERIOD'] = pd.cut(current_sensor['DATE_TIME'].dt.hour, 
                                bins=[0, 6, 12, 15, 20, 24],
                                labels=[1, 2, 3, 4, 5],
                                include_lowest=True)

            # Create dummy variables
            season_encoded = pd.get_dummies(current_sensor['SEASON'].astype(str), prefix='Season')
            time_period_encoded = pd.get_dummies(current_sensor['TIME_PERIOD'].astype(str), prefix='TimePeriod')
            current_sensor = pd.concat([current_sensor, season_encoded, time_period_encoded], axis=1)
            current_sensor = current_sensor.drop(['SEASON', 'TIME_PERIOD'], axis=1)
            
            # Prepare features and target
            X = current_sensor[['Conversion_Efficiency','Normalized_Power','Energy_Per_Irradiation','AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION'] +
                        list(season_encoded.columns) + list(time_period_encoded.columns)]
            y = current_sensor['AC_POWER']

            if len(X) > 0:  # Only append if there is data
                final_locB.append([X,y])
        else:
            final_locB.append([[],[]])

    print("*************************************","Location B: ",len(final_locB))

    # Train and evaluate models
    total_mse_A = 0
    total_r2_A = 0
    count_A = 0

    for i, sensor_data in enumerate(final_locA):
        if len(sensor_data[0]) == 0:
            locA_models.append(None)
            continue
        if sensor_data[0].shape[0] >= 5:  # Ensure enough data to split
            model, mse, r2 = train_sensor_model(sensor_data[0], sensor_data[1])
            locA_models.append(model)
            total_mse_A += mse
            total_r2_A += r2
            count_A += 1
            # print(f"Model trained for sensor {i+1}, Location: A")
            # print(f"MSE: {mse:.4f}, R2 Score: {r2:.4f}")
        else:
            # print(f"Insufficient data for model training - Sensor {i+1}, Location: A")
            locA_models.append(None)

    if count_A > 0:
        print(f"\nLocation A Overall Metrics:")
        print(f"Average MSE: {total_mse_A/count_A:.4f}")
        print(f"Average R2 Score: {total_r2_A/count_A:.4f}")

    total_mse_B = 0 
    total_r2_B = 0
    count_B = 0

    for i, sensor_data in enumerate(final_locB):
        if len(sensor_data[0]) == 0:
            locB_models.append(None)
            continue
        if sensor_data[0].shape[0] >= 5:  # Ensure enough data to split
            model, mse, r2 = train_sensor_model(sensor_data[0], sensor_data[1])
            locB_models.append(model)
            total_mse_B += mse
            total_r2_B += r2
            count_B += 1
            # print(f"Model trained for sensor {i+1}, Location: B")
            # print(f"MSE: {mse:.4f}, R2 Score: {r2:.4f}")
        else:
            # print(f"No data for Sensor {i+1}, Location: B")
            locB_models.append(None)

    if count_B > 0:
        print(f"\nLocation B Overall Metrics:")
        print(f"Average MSE: {total_mse_B/count_B:.4f}")
        print(f"Average R2 Score: {total_r2_B/count_B:.4f}")
    return locA_models,locB_models


In [4]:
def test_model(locA_models,locB_models,model_type):
    locA_solar_sensor = []
    locB_solar_sensor = []

    for i in range(1, 45):
        try:
            t = pd.read_csv(f"../Datasets/sensor_wise_data/A{i}_df.csv")
            t['DATE_TIME'] = pd.to_datetime(t['DATE_TIME'])
            locA_solar_sensor.append(t)
        except Exception as e:
            locA_solar_sensor.append([])
        try:
            t = pd.read_csv(f"../Datasets/sensor_wise_data/B{i}_df.csv")
            t['DATE_TIME'] = pd.to_datetime(t['DATE_TIME'])
            locB_solar_sensor.append(t)
        except Exception as e:
            locB_solar_sensor.append([])

    # Process Location A sensors
    for i in range(len(locA_solar_sensor)):
        if len(locA_solar_sensor[i]) > 0:
            current_sensor = locA_solar_sensor[i].copy()
            current_sensor['SEASON'] = current_sensor['DATE_TIME'].dt.month.map({
                12:1, 1:1, 2:1,
                3:2, 4:2, 5:2,
                6:3, 7:3, 8:3, 
                9:4, 10:4, 11:4})
            current_sensor['TIME_PERIOD'] = pd.cut(current_sensor['DATE_TIME'].dt.hour,
                                                    bins=[0, 6, 12, 15, 20, 24],
                                                    labels=[1, 2, 3, 4, 5],
                                                    include_lowest=True)
            # One-hot encode categorical variables
            season_encoded = pd.get_dummies(current_sensor['SEASON'].astype(str), prefix='Season')
            time_period_encoded = pd.get_dummies(current_sensor['TIME_PERIOD'].astype(str), prefix='TimePeriod')

            current_sensor = pd.concat([current_sensor, season_encoded, time_period_encoded], axis=1)
            current_sensor = current_sensor.drop(['SEASON', 'TIME_PERIOD'], axis=1)

            current_sensor.dropna(inplace=True)

            test_x = current_sensor[['Conversion_Efficiency','Normalized_Power','Energy_Per_Irradiation','AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION'] +
                        list(season_encoded.columns)]
            if locA_models[i-1] is None:
                # print(f"Model for sensor {i} is not available.")
                continue
            for period in locA_models[i-1].feature_names_in_:
                if "TimePeriod_" in period:
                    test_x = test_x.copy() 
                    test_x.loc[:,period] = current_sensor[period]
            model = locA_models[i-1]
            
            
            sample_predictions = model.predict(test_x)
            
            # Calculate differences and z-score for anomaly detection
            comparison_df = pd.DataFrame({
                'Actual AC Power': current_sensor["AC_POWER"],
                'Predicted AC Power': sample_predictions,
                'Difference': current_sensor["AC_POWER"] - sample_predictions,
            })

            # Calculate z-score of differences
            z_scores = np.abs((comparison_df['Difference'] - comparison_df['Difference'].mean()) / comparison_df['Difference'].std())

            # Flag anomalies where z-score > 3 (3 standard deviations)
            anomalies = z_scores > 3

            # Add anomaly flags to original data
            temp_idx = current_sensor.index
            current_sensor.loc[temp_idx, f'ANOMALY_SENSOR_{i}'] = anomalies
            
            if model_type == 0:
                current_sensor.to_csv(f"sensor_wise_data_updated/LinearRegression/A{i}_df.csv",index=False)
            elif model_type == 1:
                current_sensor.to_csv(f"sensor_wise_data_updated/RandomForest/A{i}_df.csv",index=False)
            elif model_type == 2:
                current_sensor.to_csv(f"sensor_wise_data_updated/GradientBoosting/A{i}_df.csv",index=False)

            print(f"\nDetected {anomalies.sum()} anomalies for sensor {i}")
            # print("\nSample anomalous readings:")

            locA_solar_sensor[i] = current_sensor
        else:
            locA_solar_sensor.append([])
    # Process Location B sensors
    for i in range(len(locB_solar_sensor)):
        if len(locB_solar_sensor[i]) > 0:
            current_sensor = locB_solar_sensor[i].copy()
            current_sensor['SEASON'] = current_sensor['DATE_TIME'].dt.month.map({
                12:1, 1:1, 2:1,
                3:2, 4:2, 5:2,
                6:3, 7:3, 8:3, 
                9:4, 10:4, 11:4})
            current_sensor['TIME_PERIOD'] = pd.cut(current_sensor['DATE_TIME'].dt.hour,
                                                    bins=[0, 6, 12, 15, 20, 24],
                                                    labels=[1, 2, 3, 4, 5],
                                                    include_lowest=True)
            # One-hot encode categorical variables
            season_encoded = pd.get_dummies(current_sensor['SEASON'].astype(str), prefix='Season')
            time_period_encoded = pd.get_dummies(current_sensor['TIME_PERIOD'].astype(str), prefix='TimePeriod')

            current_sensor = pd.concat([current_sensor, season_encoded, time_period_encoded], axis=1)
            current_sensor = current_sensor.drop(['SEASON', 'TIME_PERIOD'], axis=1)

            current_sensor.dropna(inplace=True)

            test_x = current_sensor[['Conversion_Efficiency','Normalized_Power','Energy_Per_Irradiation','AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION'] +
                        list(season_encoded.columns)]
            if locB_models[i-1] is None:
                # print(f"Model for sensor {i} is not available.")
                continue
            for period in locB_models[i-1].feature_names_in_:
                if "TimePeriod_" in period:
                    test_x = test_x.copy() 
                    test_x.loc[:,period] = current_sensor[period]
            model = locB_models[i-1]

            sample_predictions = model.predict(test_x)

            # Calculate differences and z-score for anomaly detection
            comparison_df = pd.DataFrame({
                'Actual AC Power': current_sensor["AC_POWER"],
                'Predicted AC Power': sample_predictions,
                'Difference': current_sensor["AC_POWER"] - sample_predictions
            })

            # Calculate z-score of differences
            z_scores = np.abs((comparison_df['Difference'] - comparison_df['Difference'].mean()) / comparison_df['Difference'].std())

            # Flag anomalies where z-score > 3 (3 standard deviations)
            anomalies = z_scores > 3

            # Add anomaly flags to original data
            temp_idx = current_sensor.index
            current_sensor.loc[temp_idx, f'ANOMALY_SENSOR_{i}'] = anomalies

            if model_type == 0:
                current_sensor.to_csv(f"sensor_wise_data_updated/LinearRegression/B{i}_df.csv",index=False)
            elif model_type == 1:
                current_sensor.to_csv(f"sensor_wise_data_updated/RandomForest/B{i}_df.csv",index=False)
            elif model_type == 2:
                current_sensor.to_csv(f"sensor_wise_data_updated/GradientBoosting/B{i}_df.csv",index=False)

            print(f"\nDetected {anomalies.sum()} anomalies for sensor {i}")
            # print("\nSample anomalous readings:")
            # print(comparison_df[anomalies].head())

            locB_solar_sensor[i] = current_sensor
        else:
            locB_solar_sensor.append([])
    # Plotting anomalies detected in Location A
    plt.figure(figsize=(15, 10))
    plt.suptitle('Anomalies Detected in Location A', fontsize=16)

    for i in range(len(locA_solar_sensor)):
        if len(locA_solar_sensor[i]) == 0:
            continue
        anomaly_col = f'ANOMALY_SENSOR_{i}'
        if anomaly_col in locA_solar_sensor[i].columns:
            sensor_data = locA_solar_sensor[i]
            if not sensor_data.empty and sensor_data[anomaly_col].any():
                anomalies = sensor_data[sensor_data[anomaly_col]]
                plt.scatter(anomalies['DATE_TIME'], 
                            [i] * len(anomalies), 
                            label=f'Sensor {i+1}',
                            alpha=0.6)

    plt.ylabel('Sensor Number')
    plt.xlabel('Date Time')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.tight_layout()

    # Create separate figure for Location B
    plt.figure(figsize=(15, 10))
    plt.suptitle('Anomalies Detected in Location B', fontsize=16)

    for i in range(len(locB_solar_sensor)):
        if len(locB_solar_sensor[i]) == 0:
            continue
        anomaly_col = f'ANOMALY_SENSOR_{i}'
        if anomaly_col in locB_solar_sensor[i]:
            sensor_data = locB_solar_sensor[i]
            if not sensor_data.empty and sensor_data[anomaly_col].any():
                anomalies = sensor_data[sensor_data[anomaly_col]]
                plt.scatter(anomalies['DATE_TIME'], 
                            [i] * len(anomalies), 
                            label=f'Sensor {i+1}',
                            alpha=0.6)

    plt.ylabel('Sensor Number')
    plt.xlabel('Date Time')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    

In [5]:
# Initialize lists to store processed data for each location
final_locA = []
final_locB = []
print("Linear Regression")
# Function to train model for a sensor
def train_sensor_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return model, mse, r2

# Lists to store models and their performance metrics
locA_models = []
locB_models = []
locA_models,locB_models = preprocess_data(locA_solar_sensor.copy(),locB_solar_sensor.copy())
test_model(locA_models,locB_models,0)

print("Random Forest")
# Initialize lists to store processed data for each location
final_locA = []
final_locB = []

# Function to train model for a sensor
def train_sensor_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return model, mse, r2

# Lists to store models and their performance metrics
locA_models = []
locB_models = []

locA_models,locB_models = preprocess_data(locA_solar_sensor.copy(),locB_solar_sensor.copy())
test_model(locA_models,locB_models,1)

print("Gradient Boosting")
# Initialize lists to store processed data for each location
final_locA = []
final_locB = []

# Function to train model for a sensor
def train_sensor_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return model, mse, r2

# Lists to store models and their performance metrics
locA_models = []
locB_models = []
locA_models,locB_models = preprocess_data(locA_solar_sensor.copy(),locB_solar_sensor.copy())
test_model(locA_models,locB_models,2)

print("finished")



Linear Regression
************************************* Location A:  44


C:\Users\SAHAN\AppData\Local\Temp\ipykernel_5056\3777848242.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Spring' 'Spring' 'Spring' ... 'Summer' 'Summer' 'Summer']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  current_sensor.loc[:, 'SEASON'] = current_sensor['DATE_TIME'].dt.month.map({
C:\Users\SAHAN\AppData\Local\Temp\ipykernel_5056\3777848242.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Spring' 'Spring' 'Spring' ... 'Summer' 'Summer' 'Summer']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  current_sensor.loc[:, 'SEASON'] = current_sensor['DATE_TIME'].dt.month.map({
C:\Users\SAHAN\AppData\Local\Temp\ipykernel_5056\3777848242.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas

************************************* Location B:  44

Location A Overall Metrics:
Average MSE: 986.0062
Average R2 Score: 0.9923

Location B Overall Metrics:
Average MSE: 15839.6510
Average R2 Score: 0.8606


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Season_2
- Season_3
Feature names seen at fit time, yet now missing:
- Season_Spring
- Season_Summer
